Аналитика следующая:
* Слабые места (куда уходит больше всего)
    - Топ категорий по расходам
    - Доли категорий
    - Всплески расходов по дням (rule-based + ML anomaly)
* Регулярные платежи
    - Группировка по категориям
    - Проверка на периодичность (weekly, monthly)
    - Итог: список "похоже на подписку/регулярный платёж"
* Экономия и "сколько можно отложить"
    - Политика экономии (правила и модель)
    - Расчёт `potential_savings_total`
    - Разложение экономии по категориям и кластерам
    - Итог: "если снизить X%, то можно отложить Y рублей за период N"

In [178]:
import re
from typing import List, Dict

import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

In [179]:
MCC_MAP = {
    "Топливо, АЗС": [5983, 5541, 5542],
    "Автомойка": [7542],

    "Одежда и обувь": [5691, 5137, 5611, 5661, 5621, 5651],

    "Такси и каршеринг": list(range(3351, 3442)) + [4121, 7512, 3990],

    "Фастфуд": [5814],

    "Электроника и техника": [5722, 5732, 5946],

    "Строительные материалы": [5039, 5198, 5211, 5231, 5251, 5713, 5714, 5712, 5074],
    "Товары для дома": [5200, 5718, 5719, 5021, 5261],

    "Автосервисы и запчасти": [7531, 7534, 7535, 7538, 5511, 5521, 5532, 5533],

    "Салоны красоты и СПА": [7230, 7297, 7298],

    "Спортивные магазины": [5655, 5940, 5941],

    "Фитнес": [7941, 7997],

    "Турагентства": [5962, 4722, 4723, 7032, 7033, 4411],

    "Отели": list(range(3501, 3990)) + list(range(3991, 4000)) + [7011],

    "Авиабилеты и ЖД билеты": [
        *list(range(3000, 3302)), 4511, 4582, 4304, 4415, 4418, 4011, 4112
    ],

    "Аптеки": [5122, 5912],

    "Медицинские услуги": [
        8011, 5976, 8031, 8042, 8043, 8049, 8050, 8062, 8021, 8071
    ],

    "Рестораны и кафе": [5811, 5812, 5813],

    "Театры, музеи и выставки": [7922, 7991, 7996, 7998, 7999],

    "Маркетплейсы": [5262, 5300, 7278],

    "Образование": [8211, 8220, 8241, 8244, 8249, 8299, 8351],

    "Детские товары": [5641, 5945],

    "Косметика": [5977],

    "Книги": [2741, 5192, 5942, 5994],

    "Цветы": [5193, 5992],

    "Кино": [7829, 7832, 7841],

    "Канцтовары": [5111, 5943],

    "Товары для животных": [742, 5995],  # 0742 -> 742

    "Бытовые услуги": [
        7210, 7211, 7216, 7217, 7251, 7349, 7379, 7622, 7623, 7629, 7641, 7692, 7699
    ],

    "Связь, интернет, ТВ": [4899, 4813, 4815, 4821, 4816, 4814],
}


In [180]:
def _find_header_row(df_raw: pd.DataFrame, required_cols: List[str], max_scan: int = 200) -> int:
    """
    Searches for the header row in messy bank Excel files.

    Args:
        df_raw: Raw DataFrame without headers.
        required_cols: Column name keywords to search for.
        max_scan: Max rows to scan.

    Returns:
        Index of header row.
    """
    best_i, best_score = 0, -1

    for i in range(min(len(df_raw), max_scan)):
        row = df_raw.iloc[i].astype(str).str.lower().tolist()

        score = 0
        for col in required_cols:
            for cell in row:
                if col.lower() in cell:
                    score += 1
                    break

        if score > best_score:
            best_score = score
            best_i = i

        if score >= len(required_cols) - 1:
            return i

    if best_score <= 0:
        raise ValueError("Не удалось найти строку заголовков.")

    return best_i


def load_bank_xlsx(path: str) -> pd.DataFrame:
    """
    Loads messy bank Excel file and extracts the real transaction table.

    Args:
        path: Path to Excel file.

    Returns:
        DataFrame with bank transactions.
    """
    df_raw = pd.read_excel(path, header=None)

    required = ["дата", "сум", "опис", "катег"]
    header_row = _find_header_row(df_raw, required_cols=required)

    df = pd.read_excel(path, header=header_row)
    df = df.dropna(how="all").reset_index(drop=True)

    df = df.loc[:, ~df.columns.astype(str).str.contains("^Unnamed")]

    return df


In [181]:
def extract_mcc(description: str) -> int | None:
    m = re.search(r"mcc[:\s]*([0-9]{4})", description.lower())
    if m:
        return int(m.group(1))
    return None


def classify_by_mcc(mcc: int) -> str | None:
    if mcc is None:
        return None

    for cat, codes in MCC_MAP.items():
        for code in codes:
            if isinstance(code, tuple):
                if code[0] <= mcc <= code[1]:
                    return cat
            else:
                if mcc == code:
                    return cat
    return None

In [211]:
def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Converts bank columns to standard schema:
    date, description, amount, category
    """
    detected: Dict[str, list] = {
        "date": [],
        "description": [],
        "amount": [],
        "category": []
    }

    for col in df.columns:
        name = str(col).lower()

        if "дата" in name:
            detected["date"].append(col)
        elif "опис" in name or "назнач" in name:
            detected["description"].append(col)
        elif "сум" in name or "amount" in name:
            detected["amount"].append(col)
        elif "катег" in name:
            detected["category"].append(col)

    final_map = {cols[0]: key for key, cols in detected.items() if len(cols) > 0}
    df = df.rename(columns=final_map)

    if "category" not in df.columns:
        df["category"] = "Другое"

    df = df[["date", "description", "amount", "category"]]
    df["date"] = pd.to_datetime(
        df["date"],
        errors="coerce",
        dayfirst=True
    )

    df["amount"] = pd.to_numeric(df["amount"], errors="coerce")

    df = df.dropna(subset=["date", "amount"])
    df["mcc"] = df["description"].astype(str).apply(extract_mcc)

    return df


# Поиск регулярных трат

In [183]:
def normalize_description(text: str) -> str:
    """
    Removes noise from bank descriptions to make clustering work.
    """
    text = text.lower()

    # убрать цифры, номера транзакций, телефоны
    text = re.sub(r"\d+", " ", text)

    # убрать мусор
    text = re.sub(r"[^\w\s]", " ", text)

    # убрать лишние пробелы
    text = re.sub(r"\s+", " ", text).strip()

    return text


def find_recurring_groups(df: pd.DataFrame) -> pd.DataFrame:
    """
    Finds recurring transaction groups using ML clustering.

    Returns:
        DataFrame with cluster_id column.
    """
    df = df.copy()
    df["clean_desc"] = df["description"].astype(str).apply(normalize_description)

    vectorizer = TfidfVectorizer(
        max_features=1000,
        ngram_range=(1, 2),
        min_df=2
    )

    X = vectorizer.fit_transform(df["clean_desc"])

    model = DBSCAN(
        eps=0.35,
        min_samples=3,
        metric="cosine"
    )

    df["cluster"] = model.fit_predict(X)

    return df


def aggregate_recurring(df: pd.DataFrame) -> pd.DataFrame:
    """
    Aggregates clustered transactions into recurring payment groups.
    """
    recurring = df[df["cluster"] != -1]

    grouped = (
        recurring
        .groupby("cluster")
        .agg(
            name=("description", lambda x: x.iloc[0][:60]),
            total=("amount", "sum"),
            count=("amount", "count"),
            avg=("amount", "mean"),
            first_date=("date", "min"),
            last_date=("date", "max")
        )
        .reset_index()
    )

    return grouped.sort_values("total")


def format_recurring_groups(groups: pd.DataFrame) -> str:
    text = ["🔁 РЕГУЛЯРНЫЕ ПЛАТЕЖИ:\n"]

    for _, row in groups.iterrows():
        text.append(
            f"- {row['name']} → {row['count']} раз, "
            f"в среднем {abs(row['avg']):.0f} ₽, "
            f"всего {abs(row['total']):.0f} ₽"
        )

    return "\n".join(text)


In [184]:
def category_statistics(df: pd.DataFrame) -> pd.DataFrame:
    total = df["amount"].abs().sum()

    stats = (
        df.groupby("final_category")["amount"]
        .sum()
        .reset_index()
        .rename(columns={"final_category": "category"})
    )

    stats["share"] = stats["amount"].abs() / total

    return stats


In [185]:
def detect_anomalies(df: pd.DataFrame) -> pd.DataFrame:
    """
    Detects anomalous transactions using Isolation Forest.

    Args:
        df: Normalized dataframe.

    Returns:
        DataFrame with an 'anomaly' column (1 = anomaly).
    """
    features = df[["amount"]].copy()

    scaler = StandardScaler()
    X = scaler.fit_transform(features)

    model = IsolationForest(contamination=0.05, random_state=42)
    df["anomaly"] = model.fit_predict(X)

    # -1 means anomaly → convert to 1
    df["anomaly"] = (df["anomaly"] == -1).astype(int)

    return df


In [186]:
def estimate_savings(df, recurring_groups, profile):
    """
    Estimates potential savings based on personal spending behavior.
    Uses user's own baseline instead of hardcoded categories.
    """
    savings = 0.0

    normal = profile[profile["is_abnormal_month"] == False]
    abnormal = profile[profile["is_abnormal_month"] == True]

    if len(normal) == 0 or len(abnormal) == 0:
        return 0.0

    baseline = normal.drop(columns=["is_abnormal_month"]).mean()

    # 1️⃣ Перерасход по месяцам
    for month, row in abnormal.iterrows():
        diff = row.drop("is_abnormal_month") - baseline

        for value in diff:
            if value > 0:
                # считаем, что 50% перерасхода можно оптимизировать
                savings += value * 0.5

    # 2️⃣ Регулярные платежи (подписки и сервисы)
    if len(recurring_groups) > 0:
        savings += abs(recurring_groups["total"].sum()) * 0.3

    return round(float(savings), 2)


In [187]:
def generate_advice(
        category_stats: pd.DataFrame,
        recurring: pd.DataFrame,
        anomalies: pd.DataFrame
) -> List[str]:
    """
    Generates human-readable financial advice.

    Args:
        category_stats: Category summary.
        recurring: Recurring payments.
        anomalies: Transactions with anomalies.

    Returns:
        List of advice strings.
    """
    advice = []

    for _, row in category_stats.iterrows():
        if row["share"] > 0.35:
            advice.append(
                f"Слишком большая доля расходов на категорию '{row['category']}' ({row['share'] * 100:.1f}%)"
            )

    if len(recurring) > 0:
        advice.append("Обнаружены регулярные платежи — стоит проверить подписки и сервисы")

    if anomalies["anomaly"].sum() > 0:
        advice.append("Найдены аномально крупные траты — возможно, импульсивные покупки")

    return advice


# Расшифровка «Прочих операций»

In [188]:
from sklearn.linear_model import LogisticRegression

In [189]:
def train_category_model(df: pd.DataFrame):
    """
    Trains ML model to predict categories from descriptions.
    """
    train_df = df[df["final_category"] != "Прочие операции"]

    X = train_df["description"].astype(str)
    y = train_df["final_category"]

    vectorizer = TfidfVectorizer(max_features=1000)
    X_vec = vectorizer.fit_transform(X)

    model = LogisticRegression(max_iter=1000)
    model.fit(X_vec, y)

    return model, vectorizer


In [190]:
def classify_other_operations(df: pd.DataFrame, model, vectorizer):
    """
    Reclassifies 'Прочие операции' using ML.
    """
    mask = df["category"] == "Прочие операции"

    X = vectorizer.transform(df.loc[mask, "description"].astype(str))
    preds = model.predict(X)

    df.loc[mask, "final_category"] = preds
    df.loc[~mask, "final_category"] = df.loc[~mask, "category"]

    return df


# Строим модель привычек пользователя (ML)

In [191]:
def build_user_profile(df: pd.DataFrame):
    """
    Learns user's normal monthly spending behavior.
    """
    df["month"] = df["date"].dt.to_period("M")

    pivot = (
        df.groupby(["month", "final_category"])["amount"]
        .sum()
        .unstack(fill_value=0)
    )

    # приводим к абсолютным тратам
    pivot = pivot.abs()

    scaler = StandardScaler()
    X = scaler.fit_transform(pivot)

    model = IsolationForest(contamination=0.2, random_state=42)
    scores = model.fit_predict(X)

    pivot["is_abnormal_month"] = scores == -1

    return pivot, model, scaler


In [192]:
def explain_monthly_anomalies(profile: pd.DataFrame) -> list[str]:
    advice = []

    normal = profile[profile["is_abnormal_month"] == False]
    abnormal = profile[profile["is_abnormal_month"] == True]

    if len(abnormal) == 0:
        advice.append("Ваш стиль расходов стабилен — резких сбоев не обнаружено.")
        return advice

    baseline = normal.mean()

    for month, row in abnormal.iterrows():
        diff = row.drop("is_abnormal_month") - baseline

        top = diff.sort_values(ascending=False).head(3)

        for cat, value in top.items():
            if value > 0:
                advice.append(
                    f"В {month} траты по категории '{cat}' были выше нормы на {value:.0f} ₽. "
                    "Это ключевая точка для оптимизации."
                )

    return advice


# Категории как точки в смысловом пространстве

In [193]:
CATEGORY_PROTOTYPES = {
    "Еда": "продукты, рестораны, доставка еды, кафе, супермаркеты",
    "Такси": "такси, поездки, транспорт, поездка на машине",
    "Подписки": "подписки, онлайн сервисы, стриминг, премиум доступ",
    "Связь": "мобильная связь, интернет, телефон",
    "Маркетплейсы": "онлайн покупки, интернет магазины",
    "Техника": "компьютеры, серверы, электроника, гаджеты"
}

In [215]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")


def embed(texts):
    return model.encode(texts, normalize_embeddings=True)


def build_category_vectors():
    labels = list(CATEGORY_PROTOTYPES.keys())
    texts = list(CATEGORY_PROTOTYPES.values())
    vecs = embed(texts)

    return labels, vecs


def semantic_classify(df):
    labels, cat_vecs = build_category_vectors()

    desc_vecs = embed(df["description"].astype(str).tolist())

    sims = desc_vecs @ cat_vecs.T  # cosine similarity

    best = sims.argmax(axis=1)

    df["semantic_category"] = [labels[i] for i in best]
    df["semantic_score"] = sims.max(axis=1)

    return df


FINANCE_KEYWORDS = [
    "сбп", "перевод", "перечисление", "копилка",
    "пополнение", "погашение", "внутрибанков",
    "между счетами", "перевод средств"
]


def is_money_movement(desc: str) -> bool:
    desc = desc.lower()
    return any(k in desc for k in FINANCE_KEYWORDS)


def smart_category(df):
    df = df.copy()

    df["is_money"] = df["description"].astype(str).apply(is_money_movement)

    df["mcc_category"] = df["mcc"].apply(classify_by_mcc)
    df = semantic_classify(df)

    df["final_category"] = np.where(
        df["is_money"],
        "Финансовые операции",   # ← выше всего
        np.where(
            df["category"] != "Прочие операции",
            df["category"],      # ← доверяем банку
            np.where(
                df["mcc_category"].notna(),
                df["mcc_category"],
                df["semantic_category"]
            )
        )
    )

    return df



# Итоговый Pipeline

In [216]:
def format_user_report(
        df: pd.DataFrame,
        recurring_groups: pd.DataFrame,
        anomalies: pd.DataFrame,
        savings: float,
        profile_advice: List[str]
) -> str:
    text = []

    # ----------------------------
    # 1. Куда уходят деньги
    # ----------------------------
    text.append("📊 КУДА УХОДЯТ ДЕНЬГИ\n")

    by_cat = (
        df.groupby("final_category")["amount"]
        .sum()
        .abs()
        .sort_values(ascending=False)
    )

    total = by_cat.sum()

    for cat, value in by_cat.items():
        share = value / total * 100
        text.append(f"- {cat}: {value:,.0f} ₽ ({share:.1f}%)")

    # ----------------------------
    # 2. Регулярные траты
    # ----------------------------
    text.append("\n🔁 ВАШИ РЕГУЛЯРНЫЕ ПЛАТЕЖИ\n")

    if len(recurring_groups) == 0:
        text.append("Регулярных платежей не найдено.")
    else:
        for _, row in recurring_groups.sort_values("total").iterrows():
            per_month = abs(row["total"]) / max(1, row["count"])
            text.append(
                f"- {row['name']} → {row['count']} раз, "
                f"≈ {per_month:.0f} ₽ за раз, "
                f"всего {abs(row['total']):,.0f} ₽"
            )

    # ----------------------------
    # 3. Аномальные операции
    # ----------------------------
    text.append("\n⚠️ НЕОБЫЧНЫЕ ТРАТЫ\n")

    if len(anomalies) == 0:
        text.append("Аномальных операций не обнаружено.")
    else:
        for _, row in anomalies.sort_values("amount").head(10).iterrows():
            text.append(
                f"- {row['date'].date()} | {row['description'][:50]}… → {row['amount']} ₽"
            )

    # ----------------------------
    # 4. Поведенческий анализ (ML)
    # ----------------------------
    text.append("\n🧠 АНАЛИЗ ВАШЕГО ФИНАНСОВОГО ПОВЕДЕНИЯ\n")

    for line in profile_advice:
        text.append(f"- {line}")

    # ----------------------------
    # 5. Итог по экономии
    # ----------------------------
    text.append("\n💰 ПОТЕНЦИАЛ ЭКОНОМИИ\n")
    text.append(f"Если оптимизировать выявленные привычки, можно сохранить около {abs(savings):,.0f} ₽ за этот период.")

    return "\n".join(text)


In [217]:
def run_pipeline(path: str):
    df = load_bank_xlsx(path)
    df = normalize_columns(df)

    # 1. Реальные категории
    df = smart_category(df)

    # 2. ML для "Прочих операций"
    model, vec = train_category_model(df)
    df = classify_other_operations(df, model, vec)

    # 3. Регулярные платежи
    df = find_recurring_groups(df)
    recurring_groups = aggregate_recurring(df)

    # 4. Аномалии транзакций
    df = detect_anomalies(df)
    anomalies = df[df["anomaly"] == 1]

    # 5. Поведенческая модель
    profile, _, _ = build_user_profile(df)
    profile_advice = explain_monthly_anomalies(profile)

    # 6. Экономия
    savings = estimate_savings(df, recurring_groups, profile)

    report = format_user_report(
        df,
        recurring_groups,
        anomalies,
        savings,
        profile_advice
    )

    return {
        "report": report,
        "recurring_groups": recurring_groups,
        "anomalies": anomalies,
        "profile": profile,
        "profile_advice": profile_advice,
        "savings": savings,
        "df": df
    }


In [218]:
res = run_pipeline('assets/12 июля 2025 - 11 января 2026.xlsx')
# res['recurring_groups']


In [219]:
print(res['report'])

📊 КУДА УХОДЯТ ДЕНЬГИ

- Финансовые операции: 132,536 ₽ (67.2%)
- Такси и каршеринг: 32,336 ₽ (16.4%)
- Телефон, интернет, ТВ: 8,206 ₽ (4.2%)
- Фастфуд: 6,316 ₽ (3.2%)
- Транспорт: 5,157 ₽ (2.6%)
- Подписки: 3,294 ₽ (1.7%)
- Аптеки: 2,061 ₽ (1.0%)
- Мультимедиа: 1,823 ₽ (0.9%)
- Кафе и рестораны: 1,690 ₽ (0.9%)
- Путешествия: 1,400 ₽ (0.7%)
- Маркетплейсы: 1,197 ₽ (0.6%)
- Животные: 615 ₽ (0.3%)
- Товары для дома: 327 ₽ (0.2%)
- Прочие расходы: 320 ₽ (0.2%)
- Образование: 15 ₽ (0.0%)

🔁 ВАШИ РЕГУЛЯРНЫЕ ПЛАТЕЖИ

- Категория: Перевод по СБП.Перевод C421507251027268 через Сис → 270 раз, ≈ 504 ₽ за раз, всего 136,001 ₽
- Операция по карте: 220015******4249, дата создания транзакци → 151 раз, ≈ 393 ₽ за раз, всего 59,367 ₽
- Перечисление средств в рамках услуги "Копилка для сдачи" со  → 37 раз, ≈ 79 ₽ за раз, всего 2,918 ₽
- Подписка "Альфа-Смарт" за период с 09.08.25 по 08.09.25 → 4 раз, ≈ 599 ₽ за раз, всего 2,396 ₽
- Операция по карте: 220015******4249, дата создания транзакци → 5 раз, ≈ 